## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-15-51-58 +0000,nyt,Gaza City and Surrounding Areas Officially Hit...,https://www.nytimes.com/2025/08/22/world/middl...
1,2025-08-22-15-47-23 +0000,nypost,Former Navy SEAL congressman looks back on reu...,https://nypost.com/2025/08/22/us-news/ex-navy-...
2,2025-08-22-15-47-12 +0000,bbc,Fed chair Powell boosts expectation of US rate...,https://www.bbc.com/news/articles/c5ylwyx43x4o...
3,2025-08-22-15-46-22 +0000,bbc,Russia trying to stop meeting on peace and pro...,https://www.bbc.com/news/articles/ckgjpmg4r23o...
4,2025-08-22-15-44-24 +0000,nyt,"Joe Gruters, a Trump Ally, to Lead the Republi...",https://www.nytimes.com/2025/08/22/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,59
151,new,24
0,gaza,13
663,out,13
16,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
88,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...,116
14,2025-08-22-15-32-00 +0000,wsj,The FBI on Friday searched the Maryland home o...,https://www.wsj.com/politics/policy/fbi-raids-...,115
279,2025-08-21-20-11-18 +0000,nypost,Trump pick Alina Habba is unlawfully serving a...,https://nypost.com/2025/08/21/us-news/trump-pi...,114
232,2025-08-21-23-19-00 +0000,wsj,"Alina Habba, Trump’s Top Prosecutor in New Jer...",https://www.wsj.com/us-news/law/alina-habba-ne...,114
11,2025-08-22-15-37-00 +0000,wsj,Defense Secretary Pete Hegseth has authorized ...,https://www.wsj.com/politics/policy/defense-se...,114


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
88,116,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...
227,73,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
173,61,2025-08-22-04-36-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
24,60,2025-08-22-15-10-43 +0000,nyt,Jerome Powell Suggests Fed Will Soon Cut Inter...,https://www.nytimes.com/2025/08/22/business/po...
46,56,2025-08-22-14-25-00 +0000,wsj,The Federal Reserve said it would no longer in...,https://www.wsj.com/economy/central-banking/fe...
234,56,2025-08-21-23-02-09 +0000,nypost,Israeli stats show civilian death rate in Gaza...,https://nypost.com/2025/08/21/world-news/israe...
14,51,2025-08-22-15-32-00 +0000,wsj,The FBI on Friday searched the Maryland home o...,https://www.wsj.com/politics/policy/fbi-raids-...
347,50,2025-08-21-16-19-40 +0000,nypost,Ex-Biden White House spokesman says he only sa...,https://nypost.com/2025/08/21/us-news/ex-biden...
273,47,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...
297,39,2025-08-21-18-57-02 +0000,cbc,Carney and Trump speak for the first time sinc...,https://www.cbc.ca/news/world/carney-trump-pho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
